In [1]:
#All required inputs

#Salary
my_salary = 180000

#Input Pension %s - Salary Sacrifice Scheme
my_pension_pct = 0.06
employer_pension_pct = 0.15

#Input other Salary Sacrifice
my_sal_sac_monthly = 150
my_sal_sac_annual = my_sal_sac_monthly * 12


#Tax & NI info

#Input Tax Code
my_tax_code = 1257

#Income Tax
basic_rate_upper = 37700
higher_rate_upper = 125140
tax_allowance_th = 100000
# additional_rate_taxable is any remaining taxable income over 125,140
basic_rate_pct = 0.2
higher_rate_pct = 0.4
additional_rate_pct = 0.45

#National Insurance
ni_primary_th = 12570
ni_uel_th = 50270

ni_primary_pct = 0.12
ni_uel_pct = 0.02


#Student Loan info

#Plan 1
sl_plan1_th = 22015
sl_plan1_pct = 0.09

#Plan 2
sl_plan2_th = 27295
sl_plan2_pct = 0.09

#Plan 4
sl_plan4_th = 27660
sl_plan4_pct = 0.09

#Plan 5
sl_plan5_th = 25000
sl_plan5_pct = 0.09

#Postgrad
sl_postgrad_th = 21000
sl_postgrad_pct = 0.06



In [2]:
def calc_pension(my_salary, my_pension_pct, employer_pension_pct):
    #Takes required inputs to calculate pension contributions from
    #Employee, Employer, and calculates Total
    my_pension_cont = my_salary * my_pension_pct
    employer_pension_cont = my_salary * employer_pension_pct
    total_pension_pct = my_pension_pct + employer_pension_pct
    total_pension_cont = my_salary * total_pension_pct
    
    return my_pension_cont, employer_pension_cont, total_pension_cont

In [3]:
my_pension_cont, employer_pension_cont, total_pension_cont = calc_pension(my_salary, my_pension_pct, employer_pension_pct)
pension_conts = [my_pension_cont, employer_pension_cont, total_pension_cont]
for contribution in pension_conts:
    print(contribution)

10800.0
27000.0
37800.0


In [4]:
def calc_net_of_sal_sac(my_salary, my_pension_cont, my_sal_sac_annual):
    #Calculates pre-tax income, net of salary sacrifice
    net_of_sal_sac = my_salary - (my_pension_cont + my_sal_sac_annual)
    return net_of_sal_sac

In [5]:
net_of_sal_sac = calc_net_of_sal_sac(my_salary, my_pension_cont, my_sal_sac_annual)
print(net_of_sal_sac)

167400.0


In [6]:
def calc_tax_free_allowance(net_of_sal_sac, my_tax_code, tax_allowance_th, higher_rate_upper):
    #Calculates personal tax allowance using pre-tax income, net of salary sacrifice
    #Which is reduced by £1 for every £2 earned over the tax allownace threshold
    default_tax_free_allowance = my_tax_code * 10
    print("Default tax free allowance based on tax code: ", default_tax_free_allowance)
    
    if net_of_sal_sac > tax_allowance_th:
        #Reduce tax free allowance by £1 for every £2 over higher rate th, until 0.
        delta = net_of_sal_sac - tax_allowance_th
        reduction = delta // 2
        print("Income Net of Salary Sacrifice is greater than the personal allowance threshold of: ", 
              tax_allowance_th, " by: ", reduction)
        
        if reduction < default_tax_free_allowance:
            my_tax_free_allowance = default_tax_free_allowance - reduction
            print("Reducing your personal allowance by: ", delta)
        
        else:
            my_tax_free_allowance = 0
            print("Reducing your personal allowance by: ", default_tax_free_allowance)
            
    print("Your tax free allowance is: ", my_tax_free_allowance)
    
    return my_tax_free_allowance
    

In [7]:
my_tax_free_allowance = calc_tax_free_allowance(net_of_sal_sac, my_tax_code, tax_allowance_th, higher_rate_upper)

Default tax free allowance based on tax code:  12570
Income Net of Salary Sacrifice is greater than the personal allowance threshold of:  100000  by:  33700.0
Reducing your personal allowance by:  12570
Your tax free allowance is:  0


In [8]:
def calc_actual_taxable_income(net_of_sal_sac, my_tax_free_allowance):
    actual_taxable_income = net_of_sal_sac - my_tax_free_allowance
    return actual_taxable_income

In [9]:
actual_taxable_income = calc_actual_taxable_income(net_of_sal_sac, my_tax_free_allowance)
print("You will pay tax on: ", actual_taxable_income)

You will pay tax on:  167400.0


In [10]:
#Calculate Tax
def calc_tax(my_tax_free_allowance, actual_taxable_income, basic_rate_upper, basic_rate_pct,
            net_of_sal_sac, additional_rate_pct, higher_rate_upper, higher_rate_pct):
    
    #0% Tax paid up to the tax free allownace
    print("You pay 0% tax up to your tax free allowance of: ", my_tax_free_allowance)
    
    #20% Tax up to the basic rate upper limit, after the allowance    
    if actual_taxable_income >= basic_rate_upper:
        basic_rate_tax = basic_rate_upper * basic_rate_pct
        print("You will pay ", basic_rate_pct*100, "% tax on ", basic_rate_upper, ", which is: ", basic_rate_tax)
    else:
        basic_rate_tax = actual_taxable_income * basic_rate_pct  
        print("You will pay ", basic_rate_pct*100, "% tax on ", basic_rate_upper, ", which is: ", basic_rate_tax)
    
    #40% Tax up to the higher rate upper limit - not lowered by the allowance
    my_higher_rate_lower_limit = basic_rate_upper + my_tax_free_allowance
    if net_of_sal_sac > my_higher_rate_lower_limit:
        if net_of_sal_sac <= higher_rate_upper:
            higher_rate_taxable = net_of_sal_sac - my_higher_rate_lower_limit
            higher_rate_tax = higher_rate_taxable * higher_rate_pct
            print("You will pay ", higher_rate_pct*100, "% tax on ", higher_rate_taxable, ", which is: ", higher_rate_tax)
        else:
            higher_rate_taxable = higher_rate_upper - my_higher_rate_lower_limit
            higher_rate_tax = higher_rate_taxable * higher_rate_pct
            print("You will pay ", higher_rate_pct*100, "% tax on ", higher_rate_taxable, ", which is: ", higher_rate_tax)
    
    #45% Tax on anything over the higher rate upper limit - not reduced by the allowance
    if net_of_sal_sac > higher_rate_upper:
        additional_rate_taxable = net_of_sal_sac - higher_rate_upper
        additional_rate_tax = additional_rate_taxable * additional_rate_pct
        print("You will pay ", additional_rate_pct*100, "% tax on ", additional_rate_taxable, ", which is: ",
              additional_rate_tax)

In [11]:
calc_tax(my_tax_free_allowance, actual_taxable_income, basic_rate_upper, basic_rate_pct,
            net_of_sal_sac, additional_rate_pct, higher_rate_upper, higher_rate_pct)

You pay 0% tax up to your tax free allowance of:  0
You will pay  20.0 % tax on  37700 , which is:  7540.0
You will pay  40.0 % tax on  87440 , which is:  34976.0
You will pay  45.0 % tax on  42260.0 , which is:  19017.0


In [12]:
def calc_ni(ni_primary_th, ni_uel_th, net_of_sal_sac, ni_primary_pct, ni_uel_pct):
    primary_ni_taxable = ni_uel_th - ni_primary_th
    upper_ni_taxable = net_of_sal_sac - ni_uel_th
    
    if net_of_sal_sac < ni_uel_th:
        my_ni_primary = (net_of_sal_sac - ni_primary_th) * ni_primary_pct
    else:
        my_ni_primary = primary_ni_taxable * ni_primary_pct
        my_ni_upper = upper_ni_taxable * ni_uel_pct
    
    return my_ni_primary, my_ni_upper

In [16]:
my_ni_primary, my_ni_upper = calc_ni(ni_primary_th, ni_uel_th, net_of_sal_sac, ni_primary_pct, ni_uel_pct)
print(my_ni_primary, my_ni_upper)

4524.0 2342.6


In [14]:
#Calculate Student Loan

In [15]:
#Calculate Net Pay